<img src="https://drive.google.com/uc?export=view&id=1YjAWn06OMcVhlyixBZBDnY17rnn7Otg5" width="100%">

# Arreglos de Dask

En este notebook veremos una introducción práctica al procesamiento distribuido con la librería `dask`, primero lo instalaremos:

In [ ]:
!pip install dask[complete] h5py

## **1. ¿Qué son los Arreglos de Dask?**
---

Los arreglos de `dask` son una forma de extender los arreglos de `numpy` para su manejo de forma distribuida y en grandes cantidades de datos. Básicamente es una forma de almacenar un arreglo multidimensional de gran tamaño de forma particionada como múltiples arreglos de `numpy` como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=1kGsy6RJH_XfTyVpvpWLSKe5dW3o2OJ3S" width="70%%">

Estos arreglos gestionan y coordinan múltiples arreglos de `numpy` (o arreglos bastante compatibles con `numpy`) que pueden existir en distintas máquinas o persistidos en disco.

Desde `dask` tenemos algunas de las operaciones típicas que normalmente usamos con `numpy` como:

* Operaciones aritméticas.
* Operaciones de reducción sobre ejes específicos.
* Reglas de broadcasting como en `numpy`.
* Reordenamiento de ejes.
* Slicing para selección de valores.
* Algunas funciones típicas de álgebra lineal.

No obstante, hay diversas operaciones de `numpy` que no son posibles en arreglos distribuidos dada su naturaleza (por ejemplo, ordenamientos, conversión a listas de _Python_, iteración con ciclos `for`, entre otras).

Veamos las características generales de los arreglos de `dask`, primero lo importamos:

In [1]:
import dask.array as da
import numpy as np

## **2. Creación**
---

Existen distintas funciones para la creación de arreglos de `dask`, estas las podemos dividir en tres categorías: arreglos determinísticos nativos, arreglos aleatorios nativos y arreglos externos.

### **2.1. Arreglos Determinísticos Nativos**
---

Existen dintintas funciones para la creación de arreglos de `dask` que son similares a `numpy`, por ejemplo:

* `zeros`: crea un arreglo lleno de ceros de un tamaño dado:

In [2]:
X = da.zeros((10, 10), dtype=np.float32)
X

dask.array<zeros_like, shape=(10, 10), dtype=float32, chunksize=(10, 10), chunktype=numpy.ndarray>

* `ones`: crea un arreglo lleno de unos de un tamaño dado:

In [3]:
X = da.ones((10, 10), dtype=np.float32)
X

dask.array<ones_like, shape=(10, 10), dtype=float32, chunksize=(10, 10), chunktype=numpy.ndarray>

* `eye`: crea un arreglo lleno de matrices identidad:

In [4]:
X = da.eye(5, dtype=np.float32)
X

dask.array<eye, shape=(5, 5), dtype=float32, chunksize=(5, 5), chunktype=numpy.ndarray>

* `arange`: crea un arreglo con los valores en un rango dado:

In [5]:
x = da.arange(1, 5)
x

dask.array<arange, shape=(4,), dtype=int32, chunksize=(4,), chunktype=numpy.ndarray>

* `linspace`: crea un número dado de elementos entre dos valores:

In [6]:
x = da.linspace(-1, 1, 100)
x

dask.array<linspace, shape=(100,), dtype=float64, chunksize=(100,), chunktype=numpy.ndarray>

### **2.2. Arreglos Aleatorios Nativos**
---

Al igual que `numpy` tenemos un paquete `random` que nos permite generar arreglos de forma aleatoria, por ejemplo, la función `randint` genera un arreglo con valores enteros asignados de forma aleatoria entre dos números y con un tamaño dado:

In [7]:
X = da.random.randint(1, 5, size=(10, 10))
X

dask.array<randint, shape=(10, 10), dtype=int32, chunksize=(10, 10), chunktype=numpy.ndarray>

De la misma forma, podemos generar números con una distribución uniforme:

In [8]:
X = da.random.uniform(-1, 1, size=(100, 100))
X

dask.array<uniform, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

Incluso arreglos con distribución normal:

In [9]:
X = da.random.normal(loc=0, scale=1, size=(100, 100))
X

dask.array<normal, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

Puede usar otras distribuciones aleatorias de `dask` como: `binomial`, `pareto`, `geometric` y muchas más, recuerde que son exactamente las mismas que están en `numpy` y las puede consultar en [este enlace](https://numpy.org/doc/stable/reference/random/index.html).

### **2.3. Arreglos Externos**
---

Podemos crear arreglos de `dask` desde distintas fuentes (como `numpy`), esto se consigue usando la función `from_array` de `dask`, veamos un ejemplo:

In [ ]:
X = da.from_array(np.random.normal(size=(100, 10)), chunks=(10, 10))
X

En este caso, usamos el parámetro `chunks` para especificar cómo serán internamente las particiones del arreglo. Recuerde que los arreglos de `dask` no están directamente en la memoria RAM hasta que no se especifique, de hecho, todos los arreglos que hemos creado hasta este momento son elementos conocidos como **promesas** (son valores que aún no se ejecutan pero esperamos que sigan determinados criterios como tipo, forma, tamaño, entre otras).

Veamos cuál es el porcentaje de utilización de la memoria RAM antes de crear un arreglo de `dask` con la librería `psutil`:

In [ ]:
import psutil
print(f"RAM: {psutil.virtual_memory()[2]}%")

Ahora, creamos un arreglo de `dask` de un gran tamaño:

In [ ]:
X = da.random.normal(size=(1_000_000, 100), chunks=(1000, 100))
X

Veamos el porcentaje de utilización de la memoria RAM luego de crear el arreglo:

In [ ]:
print(f"RAM: {psutil.virtual_memory()[2]}%")

Como puede ver, el cambio en la memoria es mínimo (el arreglo no se ha calculado aún y tampoco existe en la memoria).

Un ejemplo típico para el manejo de grandes cantidades de datos desde `dask` es con el formato `hdf5`. Se trata de un formato de archivo que es bastante compacto y que facilita la **lectura aleatoria** (no necesitamos cargar todo el archivo para extraer un valor, podemos hacerlo directamente). Podemos exportar el arreglo directamente al disco con el método `to_hdf5` sin la necesidad de que el archivo esté en la memoria:

In [ ]:
X.to_hdf5("array.h5", "/data")

Corroboremos la memoria RAM:

In [ ]:
print(f"RAM: {psutil.virtual_memory()[2]}%")

En este caso, `dask` trabaja únicamente sobre _chunks_ y no sobre el arreglo completo, por ello, únicamente es necesario tener en memoria _chunks_ de tamaño `(1000, 100)` en lugar de un arreglo de tamaño `(1000000, 100)`.

Veamos que se creó un archivo con los datos del arreglo masivo de `dask`:

In [ ]:
import os
size = os.stat("array.h5").st_size / (1024 ** 2)
print(f"Tamaño del archivo {size:.2f}MB")

De la misma forma, podemos cargar un arreglo masivo a partir de archivos `hdf5` desde `dask` con la librería `h5py`:

In [ ]:
import h5py
file = h5py.File("array.h5", "r")
data = file["data"]
data

Lo cargamos con `dask`:

In [ ]:
X = da.from_array(data, chunks=(1000, 100))
X

Finalmente, cerramos el archivo:

In [ ]:
file.close()

## **3. Atributos**
---

Los arreglos de `dask` tienen atributos adicionales a los arreglos de `numpy`, veamos los más útiles:

El atributo `shape` nos permite extraer el tamaño del arreglo:

In [ ]:
print(X.shape)

El atributo `dtype` nos permite saber qué tipo de datos contiene el arreglo:

In [ ]:
print(X.dtype)

El atributo `chunksize` permite extraer el tamaño de las particiones del arreglo:

In [ ]:
print(X.chunksize)

También podemos extraer el número completo de elementos que hay en el arreglo con el atributo `size`:

In [ ]:
print(X.size)

## **4. Métodos**
---

Los arreglos de `dask` usan métodos similares a los de un arreglo de `numpy` y algunos adicionales que nos van a permitir controlar el manejo de memoria. Veamos algunos ejemplos:

* `compute`: este método permite evaluar el arreglo de `dask` y obtener el arreglo de `numpy` correspondiente, veamos un ejemplo:

In [ ]:
X = da.random.uniform(size=(100, 10), chunks=(10, 10))
X

Usamos el método `compute`:

In [ ]:
X_np = X.compute()
print(X_np)

* `rechunk`: permite cambiar el tamaño de los _chunks_ de un arreglo, por ejemplo:

In [ ]:
X2 = X.rechunk(chunks=(50, 10))
X2

* `persist`: permite guardar en disco los resultados de una operación:

In [ ]:
X3 = X2.persist()
X3

Adicional a esto, tenemos métodos similares como los que manejan los arreglos de `numpy`, veamos algunos:

* `sum` permite sumar los elementos de un arreglo sobre un eje dado:

In [ ]:
sums = X3.sum(axis=1)
sums

Evaluamos el resultado:

In [ ]:
sums.compute()

* `mean` permite promediar los elementos de un arreglo sobre un eje dado:

In [ ]:
means = X3.mean(axis=1)
means

Evaluamos el resultado:

In [ ]:
print(means.compute())

* `std` permite calcular la desviación estándar de los elementos de un arreglo sobre un eje dado:

In [ ]:
stds = X3.std(axis=1)
stds

Evaluamos el resultado:

In [ ]:
print(stds.compute())

* `reshape` premite cambiar la forma de un arreglo:

In [ ]:
X4 = X3.reshape((10, 10, 10))
print(X4)

Debe tener cuidado con la operación de `reshape` ya que esta únicamente permite dividir o combinar ejes.

También podemos aplicar la operación `flatten` para cambiar la forma del arreglo

In [ ]:
X5 = X4.flatten()
X5

## **5. Operaciones con Arreglos**
---

Podemos realizar distintos tipos de operaciones aritméticas, geométricas y algebraicas sobre los arreglos de `dask`.

Por ejemplo, podemos sumar dos arreglos, comenzamos definiéndolos:

In [ ]:
X1 = da.random.normal(loc=0, scale=1, size=(10, 10))
X1

In [ ]:
X2 = da.random.normal(loc=0, scale=1, size=(10, 10))
X2

La suma se realiza elemento a elemento:

In [ ]:
X3 = X1 + X2
X3

Veamos el resultado:

In [ ]:
print(X3.compute())

Lo mismo aplica para operaciones como resta:

In [ ]:
X3 = X1 - X2
X3

La multiplicación elemento a elemento:

In [ ]:
X3 = X1 * X2
X3

La división:

In [ ]:
X3 = X1 / X2
X3

Podemos realizar un producto matricial con el operador `@`:

In [ ]:
X3 = X1 @ X2
X3

También podemos aplicar las reglas de _broadcasting_ como se usan en `numpy`, por ejemplo, para convertir a ceros todas las columnas pares:

In [ ]:
mask = (da.arange(10) % 2).reshape(-1, 1)
mask

Aplicamos un producto elemento a elemento para enmascarar:

In [ ]:
res = X1 * mask
res

Veamos el resultado:

In [ ]:
print(res.compute())

También podemos aplicar distintas funciones de forma distribuida de `dask`, por ejemplo la raíz cuadrada:

In [ ]:
X3 = da.sqrt(X1)
X3

El exponente:

In [ ]:
X3 = da.exp(X1)
X3

El logaritmo:

In [ ]:
X3 = da.log(X1)
X3

El módulo `linalg` de `dask` nos permite realizar algunas operaciones algebraicas que pueden ser distribuidas facilmente, entre ellas tenemos:

* Inverso de una matriz con `inv`.
* Métodos de descomposición matricial como `cholesky`, `svd`, `lu`, `qr`, entre otras.
* Normas de vectores `norm`.

Veamos un ejemplo donde calculamos una matriz al cuadrado, calculamos el inverso de la matriz y calculamos la norma de sus vectores con respecto a su primer eje:

In [ ]:
X = da.random.normal(size=(10, 5))
X2 = X @ X.T + da.eye(10) # .T nos permite transponer la matriz.
X3 = da.linalg.inv(X2)
X4 = da.linalg.norm(X3, axis=0)
print(X4.compute())

Podemos concatenar arreglos de la misma forma que en un arreglo de `numpy` con la función `concatenate`, veamos un ejemplo con los siguientes arreglos:

In [ ]:
X1 = da.random.normal(size=(20, 10), chunks=(10, 10))
X1

In [ ]:
X2 = da.random.normal(size=(20, 20), chunks=(10, 20))
X2

Veamos la concatenación:

In [ ]:
X3 = da.concatenate([X1, X2], axis=1)
X3

## **6. Indexación**
---

Podemos indexar los arreglos de `dask` siguiendo la misma notación que usamos en los arreglos de `numpy`, veamos un ejemplo sobre el siguiente arreglo:

In [ ]:
X = da.random.normal(size=(100, 10), chunks=(10, 10))
X

Podemos seleccionar la primer fila:

In [ ]:
x_0 = X[0]
x_0

Con `:` podemos seleccionar la primer columna:

In [ ]:
x_0 = X[:, 0]
x_0

También podemos seleccionar rangos valores específicos, por ejemplo, las dos primeras filas y las dos primeras columnas:

In [ ]:
X2 = X[:2, :2]
X2

De igual forma, podemos seleccionar valores específicos dando una lista de valores, por ejemplo, seleccionamos las filas 0, 3 y 5:

In [ ]:
X2 = X[[0, 3, 5]]
X2

También podemos aplicar selección condicional:

In [ ]:
X2 = X[X > 0]
X2

Evaluemos el resultado:

In [ ]:
print(X2.compute())

Como puede ver, los arreglos de `dask` son una herramienta poderosa que nos permite escalar los arreglos de `numpy` a grandes cantidades de datos y su ejecución de forma distribuida. No obstante, si estamos manejando un conjunto de datos pequeño no es muy recomendable usar `dask`, veamos una comparativa entre las operaciones con `dask` y con `numpy` con una matriz pequeña de `(1000, 1000)` usando la librería `time`:

In [ ]:
import time

Vamos a implementar un producto matricial desde `numpy` primero y a evaluar el tiempo:

In [ ]:
t0 = time.time()
X = np.random.uniform(size=(1000, 1000))
Y = X @ X
delta_t = time.time() - t0
print(delta_t)

Ahora veamos la misma operación con `dask`:

In [ ]:
t0 = time.time()
X = da.random.uniform(size=(1000, 1000), chunks=(100, 100))
Y = (X @ X).compute()
delta_t = time.time() - t0
print(delta_t)

Como puede ver, la operación con `dask` puede tomar más tiempo. Esto se debe a que `dask` tiene que coordinar y manipular las operaciones entre varios arreglos y luego unir los resultados. Esto tiene beneficios con grandes cantidades de datos pero genera un cuello de botella con datasets pequeños.

## **7. Recursos Adicionales**
---

* [Arreglos de Dask](https://docs.dask.org/en/stable/array.html).
* [Dask - Talks & tutorials](https://docs.dask.org/en/stable/presentations.html).

## **8. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*